In [10]:
from virp import Session
Session(supercell=[3,3,3], no_of_samples = 2)

CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
Processing .cif file: _disordered_cifs\Verkelis.cif
Directory created at: Verkelis
Supercell created and saved as  Verkelis\Verkelis_supercell.cif
Generating virtual cell # 0 :
Disordered site name:  In1
- Number of elements in this site:  1
- Number of sites in supercell:  108
- Atoms and site assignment (float/rounded):  [('In', 72.36, 72)]
- No of filled sites:  72 / 108
Generating virtual cell # 1 :
Disordered site name:  In1
- Number of elements in this site:  1
- Number of sites in supercell:  108
- Atoms and site assignment (float/rounded):  [('In', 72.36, 72)]
- No of filled sites:  72 / 108
All cells generated (see _JOBDONE file).
Results saved to virp_session_summary.csv


In [4]:
from pathlib import Path
for filename in Path("_disordered_cifs/").glob("*.cif"): print(filename)

_disordered_cifs\HEA.cif


In [7]:
from virp.database import is_SiteDisordered
import warnings
warnings.simplefilter("ignore")
is_SiteDisordered("_disordered_cifs\\HEA.cif")

{'has_partial': True,
 'partial_sites': [(0, 'H0+', 0.75),
  (1, 'H0+', 0.75),
  (2, 'Zr0+', 0.2),
  (2, 'Ti0+', 0.2),
  (2, 'Hf0+', 0.2),
  (2, 'Mo0+', 0.2),
  (2, 'Ta0+', 0.2)]}

In [1]:
from poshcar.disorder import VirtualLibrary, cif2vasp_occ
from sklearn.linear_model import LinearRegression
from pymatgen.core.structure import Structure
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path

# Loop over files in session
disordered_cifs = Path("_disordered_cifs")

# Function to format chemical formula with subscripts
def format_formula(formula):
    return ''.join([f'{char}' if not char.isdigit() else f'$_{{{char}}}$' for char in formula])

for cif_file in disordered_cifs.glob("*.cif"):
    header = cif_file.stem  # Extract filename without extension
    jobdone_path = Path(header) / "_JOBDONE"

    # Proceed only if _JOBDONE exists
    if jobdone_path.exists():
        structure = Structure.from_file(cif_file)
        chemical_formula = structure.composition.reduced_formula
        formatted_formula = format_formula(chemical_formula)

        targetdata = cif2vasp_occ(str(cif_file), verbose=False)

        for branch in ["stropt", "no_stropt"]:
            branch_path = Path(header) / branch
            datalist, summary_df = VirtualLibrary(branch_path, targetdata, pauling_weight=1, structure_opt=False, verbose=False)
            summary_df.to_csv(branch_path / "connectivity.csv")

            # Plotting segment
            x = np.array(summary_df['BondDiff']).reshape(-1, 1)
            y = np.array(summary_df['FormationEnergy'])

            # Perform linear regression
            model = LinearRegression()
            model.fit(x, y)
            y_pred = model.predict(x)

            # Calculate R² value
            r_sq = model.score(x, y)

            # Sort data for smooth plotting
            sorted_indices = np.argsort(summary_df['BondDiff'])
            x_sorted = np.array(summary_df['BondDiff'])[sorted_indices]
            y_pred_sorted = y_pred[sorted_indices]

            # Scatterplot with best-fit line
            plt.figure(figsize=(4, 4))
            plt.scatter(summary_df['BondDiff'], summary_df['FormationEnergy'], color='blue', alpha=0.7, label='Data')
            plt.plot(x_sorted, y_pred_sorted, color='grey', linestyle='--', linewidth=2, label=f'Fit (R²={r_sq:.2f})', zorder=2)

            # Labels and title
            plt.xlabel('Bonding deviation ΔB$_{{{P5}}}$', fontsize=14)
            plt.ylabel('Formation Energy (eV/at.)', fontsize=14)
            plt.title(f'{formatted_formula}', fontsize=16)

            # Display R² value on the plot
            plt.text(0.05, 0.9, f"R² = {r_sq:.2f}", fontsize=14, color='black', transform=plt.gca().transAxes)

            # Grid and legend
            plt.grid(True, linestyle='--', alpha=0.5)
            plt.legend()

            # Save the plot as a .png file
            plt.savefig(branch_path / "scatterplot.png", dpi=300, bbox_inches='tight')
            plt.close()


C:\Users\andyp\anaconda3\envs\Vaspnest\lib\site-packages\pymatgen\core\structure.py:3116: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:
C:\Users\andyp\anaconda3\envs\Vaspnest\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
C:\Users\andyp\anaconda3\envs\Vaspnest\lib\site-packages\pymatgen\core\structure.py:3116: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:
C:\Users\andyp\anaconda3\envs\Vaspnest\lib\site-packages\pymatgen\io\vasp\inputs.py:631: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="wt") as f

Reading from file: _disordered_cifs\Verkelis.vasp
CHGNet v0.3.0 initialized with 412,525 parameters
CHGNet will run on cpu
Site-disordered structure detected!
Site-disordered structure detected!
Unaverageness:  0.0
Reading from file: Verkelis\stropt\Verkelis_virtual_0_stropt.vasp


C:\Users\andyp\anaconda3\envs\Vaspnest\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Unaverageness:  34.09693827897162
Bonding matrix distance:  3.5131285421451794  Unaverageness:  34.09693827897162


C:\Users\andyp\anaconda3\envs\Vaspnest\lib\site-packages\pymatgen\core\structure.py:3116: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:
C:\Users\andyp\anaconda3\envs\Vaspnest\lib\site-packages\pymatgen\io\vasp\inputs.py:631: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="wt") as file:


...

Unaverageness:  34.271871848499906
Bonding matrix distance:  3.6037033420882163  Unaverageness:  34.271871848499906


C:\Users\andyp\anaconda3\envs\Vaspnest\lib\site-packages\pymatgen\core\structure.py:3116: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="rt", errors="replace") as file:
C:\Users\andyp\anaconda3\envs\Vaspnest\lib\site-packages\pymatgen\io\cif.py:1290: UserWarning: Issues encountered while parsing CIF: 77 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))
C:\Users\andyp\anaconda3\envs\Vaspnest\lib\site-packages\pymatgen\io\vasp\inputs.py:631: EncodingWarning: We strongly encourage explicit `encoding`, and we would use UTF-8 by default as per PEP 686
  with zopen(filename, mode="wt") as file:


Reading from file: Verkelis\no_stropt\Verkelis_virtual_1.vasp


C:\Users\andyp\anaconda3\envs\Vaspnest\lib\site-packages\numpy\core\fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)


Unaverageness:  34.33017914313877
Bonding matrix distance:  3.6037033420882163  Unaverageness:  34.33017914313877
Summary of results written to _operations/VirtualLibrary.csv
